In [1]:
import importlib
importlib.reload(prep)
importlib.reload(pw)

NameError: name 'prep' is not defined

In [1]:
import features_nodewise as nw
import features_pairwise as pw
import preprocessing as prep

import importlib
# import matplotlib.pyplot as plt
import random
import numpy as np
import igraph
from sklearn import preprocessing
import nltk
import csv
from functools import reduce
from sklearn import metrics
# from nltk.tokenize import RegexpTokenizer
import pickle

# Read train / test node pairs
nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("./train_train_set.csv", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

with open("./train_test_set.csv", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)
    
with open("./testing_set.csv", "r") as f:
    reader = csv.reader(f)
    competition_set  = list(reader)
    
with open("./rf_predictions.csv", "r") as f:
    reader = csv.reader(f)
    competition_predictions  = list(reader)
    
competition_set = [element[0].split(" ") for element in competition_set]

random.seed(0)
training_set = [element[0].split(" ") for element in training_set]
to_keep_train = random.sample( range(len(training_set)),k=int(round(len(training_set)*0.05)) )
training_set_reduced = [training_set[i] for i in to_keep_train]

testing_set = [element[0].split(" ") for element in testing_set]
to_keep_test = random.sample( range(len(testing_set)),k=int(round(len(testing_set)*1)) )
testing_set_reduced = [testing_set[i] for i in to_keep_test]

with open("./node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)
    
# corpus = [element[5] for element in node_info]
# titles = [element[2] for element in node_info]

# t_titles = prep.tfidf(titles)
# t = prep.tfidf(corpus)
# l = nw.LSA(t,n_components=100)

# t_ngrams = prep.tfidf(corpus, r= (2,3), midf = 2, madf=0.5,feats=150000, sublinear = True)
# l_ngrams = nw.LSA(t_ngrams,n_components=300)

# kdtree = nw.KDTree(l)
# kdtree_n = nw.KDTree(l_ngrams)

# IDs = [element[0] for element in node_info]
# node_dict = prep.to_dict( [element[0] for element in node_info],range(len(node_info)) )
# index_dict = prep.to_dict(range(len(IDs)),IDs)

[nltk_data] Downloading package punkt to /home/jacob_pfau/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jacob_pfau/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
title_sim = t_titles*t_titles.transpose()

In [2]:
import copy

competition_set_withpreds = copy.deepcopy(competition_set)
for i in range(len(competition_set)):
    competition_set_withpreds[i].append(competition_predictions[i+1][1])
competition_set_withpreds = np.array(competition_set_withpreds)

all_set = np.concatenate((training_set,testing_set))
all_set = np.concatenate((all_set,competition_set_withpreds))

In [3]:
import copy
# create graphs (train vs test), (reduced vs full)
# for some reason np.mean([nw.node_degree(t[0],gold_train_graph) for t in random.sample(training_set,10000)]) is less than mean for competition graph?

train_IDs = np.array(node_info)[:,0] 
# train_edges = [(element[0],element[1]) for element in training_set]
# train_graph = prep.article_graph(train_IDs,train_edges)
train_edges = [(element[0],element[1]) for element in training_set if element[2]=='1']
train_graph = prep.article_graph(train_IDs,train_edges)

#test graph is train graph
# test_IDs = train_IDs
# test_edges = train_edges
# test_graph = prep.article_graph(test_IDs,test_edges)

competition_IDs = train_IDs
competition_edges = train_edges
competition_edges.extend([(element[0],element[1]) for element in testing_set if element[2]=='1'])
competition_edges.extend([(element[0],element[1]) for element in competition_set_withpreds if element[2]=='1'])
competition_graph = prep.article_graph(competition_IDs,competition_edges)

In [4]:
## Read all the features that we have stored in files
import os.path

def to_feature_shape(feat):
    feat = np.array(feat)
    if len(feat.shape) == 1:#not a real array but just a long list
        feat = np.reshape(feat,(feat.shape[0],1))
    return feat

#This method should throw an error if something goes wrong
def read_feature(file_path):
    f = open(file_path, 'rb')
    feat = to_feature_shape(pickle.load(f))
    f.close()
    return feat
    
features_to_read = ["overlap_title",
                    "comm_auth",
                    "temp_diff",
                    "edge_check",
                    "LSA_distance",
                    "title_sim",
                    "N_LSA_distance",
                    "citation_check",
                    "max_sim",
                    "peer_popularity",
                    "temporal_fit",
                    "succ_pred",
                    "node_degree",
                   "path_length",
                   "Reverse_Max_Sim"]



train_features_dict = dict()
train_features_reduced_dict = dict()
test_features_dict = dict()
competition_features_dict = dict()

for name in features_to_read:
    # Train
    loaded = False
#     file_path = './features_train/'+name+'_goldgraphfixed'
# #     if name != 'edge_check' and name!='max_sim' and name!='succ_pred' and name!='citation_check':
#     try:
#         this_feat = read_feature(file_path)
#         train_features_dict[name] = this_feat
#         train_features_reduced_dict[name] = this_feat[to_keep_train,:]
#         loaded = True
#         print(1)
#     except:
#         pass
        
#     if loaded==False:
#         # Train reduced
#         file_path = './features_train/'+name+'_reducedgraph'
#         try:
#             this_feat = read_feature(file_path)
#             train_features_dict[name] = this_feat
#             train_features_reduced_dict[name] = this_feat[to_keep_train,:]
#             loaded = True
#         except:
#             pass
        
#     if loaded==False:
#         # Train reduced
#         file_path = './features_train/'+name
#         try:
#             this_feat = read_feature(file_path)
#             train_features_dict[name] = this_feat
#             train_features_reduced_dict[name] = this_feat[to_keep_train,:]
#             loaded = True
#         except:
#             pass
        
#     loaded = False
#     # Test
# #     if name != 'edge_check' and name!='max_sim' and name!='succ_pred' and name!='citation_check':
#     file_path = './features_test/'+name+'_goldgraphfixed'
#     try:
#         this_feat = read_feature(file_path)
#         test_features_dict[name] = this_feat
#         loaded = True
#         print(2)
#     except:
#         pass
    
#     if loaded == False:
#         # Test reduced
#         file_path = './features_test/'+name+'_reducedgraph'
#         try:
#             this_feat = read_feature(file_path)
#             test_features_dict[name] = this_feat
#             loaded = True
#         except:
#             pass
    
#     if loaded == False:
#         # Test reduced
#         file_path = './features_test/'+name
#         try:
#             this_feat = read_feature(file_path)
#             test_features_dict[name] = this_feat
#         except:
#             pass
        
    
    
    file_path = './'+name+'_allgraph'
    try:
        this_feat = read_feature(file_path)
        competition_features_dict[name] = this_feat
    except:
        print('failed')
    # Test reduced
#     file_path = './features_competition/'+name+'_reducedgraph'
#     try:
#         this_feat = read_feature(file_path)
#         competition_features_dict[name] = this_feat
#     except:
#         pass
    
#     file_path = './features_competition/'+name+'_goldgraphfixed'
#     try:
#         this_feat = read_feature(file_path)
#         competition_features_dict[name] = this_feat
#     except:
#         pass

In [ ]:
np.reshape(train_true_labels,(train_true_labels.shape[0],))

In [5]:
import pickle
import numpy as np

train_true_labels = read_feature('./train_true_labels')
train_true_labels = np.reshape(train_true_labels,(train_true_labels.shape[0],))
train_true_labels_reduced = train_true_labels[to_keep_train]
test_true_labels = read_feature('./test_true_labels')
test_true_labels = np.reshape(test_true_labels,(test_true_labels.shape[0],))
test_true_labels_reduced = test_true_labels[to_keep_test]

all_true_labels = []
for triple in training_set:
    all_true_labels.append(triple[2])
for triple in testing_set:
    all_true_labels.append(triple[2])

In [6]:
############################################
###   Write features to disk - Training  ###
############################################
import os
import pickle

def write_feature_to_disk(feat,file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(feat,file)

print("Current wd: ",os.getcwd())
# write_feature_to_disk(train_features_reduced_dict['title_sim'],'./features_train/title_sim')

Current wd:  /home/jacob_pfau


In [6]:
#Combine all features to one vector
train_features_dict.keys()
test_features_dict.keys()
for key,feat in train_features_dict.items():
    print(key,feat.shape)
# train_features = np.concatenate([train_features_dict['overlap_title'],
#                                          train_features_dict['comm_auth'],
#                                          train_features_dict['temp_diff'],
#                                          train_features_dict['citation_check'],
#                                          train_features_dict['max_sim'],
#                                          train_features_dict['peer_popularity'],
#                                          train_features_dict['edge_check'],
#                                          train_features_dict['succ_pred'],
#                                          train_features_dict['LSA_distance'],
#                                          train_features_dict['title_sim'],
#                                          train_features_dict['temporal_fit'],
#                                         train_features_dict['N_LSA_distance'],
#                                         train_features_dict['path_length'],
#                                         train_features_dict['node_degree']]                                            
#                                         ,axis = 1)

# train_features_reduced = np.concatenate([train_features_reduced_dict['overlap_title'],
#                                          train_features_reduced_dict['comm_auth'],
#                                          train_features_reduced_dict['temp_diff'],
#                                          train_features_reduced_dict['citation_check'],
#                                          train_features_reduced_dict['max_sim'],
#                                          train_features_reduced_dict['peer_popularity'],
#                                          train_features_reduced_dict['edge_check'],
#                                          train_features_reduced_dict['succ_pred'],
#                                          train_features_reduced_dict['LSA_distance'],
#                                          train_features_reduced_dict['title_sim'],
#                                          train_features_reduced_dict['temporal_fit'],
#                                         train_features_reduced_dict['N_LSA_distance'],
#                                         train_features_reduced_dict['path_length'],
#                                         train_features_reduced_dict['node_degree']]
#                                         ,axis = 1)

# test_features = np.concatenate(        [test_features_dict['overlap_title'],
#                                         test_features_dict['comm_auth'],
#                                         test_features_dict['temp_diff'],
#                                         test_features_dict['citation_check'],
#                                         test_features_dict['max_sim'],
#                                         test_features_dict['peer_popularity'],
#                                         test_features_dict['edge_check'],
#                                         test_features_dict['succ_pred'],
#                                         test_features_dict['LSA_distance'],
#                                         test_features_dict['title_sim'],
#                                         test_features_dict['temporal_fit'],
#                                        test_features_dict['N_LSA_distance'],
#                                        test_features_dict['path_length'],
#                                        test_features_dict['node_degree']]
#                                         ,axis = 1)


competition_features = np.concatenate(  [competition_features_dict['overlap_title'],
                                        competition_features_dict['comm_auth'],
                                        competition_features_dict['temp_diff'],
                                        competition_features_dict['citation_check'],
                                        competition_features_dict['max_sim'],
                                        competition_features_dict['peer_popularity'],
                                        competition_features_dict['edge_check'],
                                        competition_features_dict['succ_pred'],
                                        competition_features_dict['LSA_distance'],
                                        competition_features_dict['title_sim'],
                                        competition_features_dict['temporal_fit'],
                                        competition_features_dict['N_LSA_distance'],
                                        competition_features_dict['path_length'],
                                        competition_features_dict['node_degree'],
                                        competition_features_dict['Reverse_Max_Sim']]
                                        ,axis = 1)

In [136]:
from sklearn.preprocessing import PolynomialFeatures as PF

poly = PF(degree=2, interaction_only=False)

selection = [1, 7, 9, 10, 11, 15, 16, 19, 21, 22, 23, 6, 25, 26, 27]
# selection = list(range(train_features_reduced.shape[1]))

train_sp = poly.fit_transform(train_features[:,selection])
test_sp = poly.fit_transform(test_features[:,selection])


In [211]:
from sklearn.preprocessing import StandardScaler as SS
scaler = SS()
normalized_competition_features = scaler.fit_transform(competition_features)
# normalized_test_features = scaler.fit_transform(test_features)

In [8]:
l_competition = len(competition_set)
l_test = len(testing_set)
reserved = l_competition+30000
label_reserved = 30000

In [213]:
from sklearn import svm
import copy

classifier = svm.LinearSVC(loss='hinge')
# selection = [23, 1, 16, 20, 22, 6, 21]
selection = list(range(train_features_reduced.shape[1]))
svm_s = copy.copy(selection)

# classifier.fit(train_features_reduced[:,selection], train_true_labels_reduced)
# preds_svm = list(classifier.predict(test_features[:,selection]))
classifier.fit(normalized_competition_features[:-reserved,selection], all_true_labels[:-reserved])
svm_c = classifier
preds_svm = list(classifier.predict(normalized_competition_features[-reserved:-l_competition,selection]))

acc = metrics.accuracy_score(list(map(int,all_true_labels[-reserved:-l_competition])), list(map(int,preds_svm)))
f1 = metrics.f1_score(list(map(int,all_true_labels[-reserved:-l_competition])), list(map(int,preds_svm)))
print('acc:',acc,'f1:',f1)

NameError: name 'train_features_reduced' is not defined

In [95]:
from sklearn.linear_model import LogisticRegression as lr 
selection = [ 1,  7,  9, 10, 11, 15, 16, 19, 21, 22, 23]
selection.extend([6,25,26,27])
selection.append(31)
lr_s = copy.copy(selection)
# selection = [i for i in range(16)]
# selection.extend([41,103,115])

model = lr(penalty='l1').fit(train_features_reduced[:,selection], train_true_labels_reduced[:])
lr_c = model
preds_lg = list(model.predict(test_features[:,selection]))

# model = lr(penalty='l1').fit(train_features_reduced, train_true_labels)
# preds_lg = list(model.predict(test_features))

acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_lg)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_lg)))
print('acc:',acc,'f1:',f1)

/home/jacob/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


acc: 0.9649333333333333 f1: 0.9672028931288191


In [ ]:
np.argsort(list(map(lambda x: abs(x),model.coef_)))[0][:14]
# sorted(list(map(lambda x: abs(x),model.coef_))[0],reverse=True)
[poly.powers_[i,:] for i in [31,28,9,17,13]]

In [15]:
from sklearn.neighbors import KNeighborsClassifier
selection = [1, 2, 6, 13, 16, 20, 21, 22]
knn_s = copy.copy(selection)

nNhbr = KNeighborsClassifier(n_neighbors=9,weights='distance')
nNhbr.fit(train_features_reduced[:,selection],train_true_labels_reduced) # do Ytrain.ravel() for length one Y values
knn_c = nNhbr
preds_knn = nNhbr.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_knn)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_knn)))
print('acc:',acc,'f1:',f1)

acc: 0.9581021178788376 f1: 0.9613848202396804


In [19]:
from sklearn.tree import DecisionTreeClassifier
# selection = [0, 1, 2, 8, 13, 16, 20, 22]
selection = [ 0,  2,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 19, 20, 21, 22,23, 25, 26, 27]
dt_s = copy.copy(selection)

dTree = DecisionTreeClassifier()
dTree.fit(train_features_reduced[:,selection],train_true_labels_reduced) # do Ytrain.ravel() for length one Y values
dt_c = dTree

preds_dt = dTree.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_dt)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_dt)))
print('acc:',acc,'f1:',f1)

acc: 0.9557872270563126 f1: 0.9595274951532184


In [57]:
test = np.concatenate((probs_lr.reshape(-1,1),probs_dt.reshape(-1,1)),axis=1)
np.concatenate((test,probs_dt.reshape(-1,1)),axis=1)

array([[2.50986568e-06, 0.00000000e+00, 0.00000000e+00],
       [9.99997490e-01, 1.00000000e+00, 1.00000000e+00],
       [9.43410527e-01, 1.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [9.24313612e-01, 1.00000000e+00, 1.00000000e+00],
       [7.56863879e-02, 0.00000000e+00, 0.00000000e+00]])

In [16]:
probs_svm = svm_c.decision_function(normalized_test_features[:,svm_s])
probs_lr = lr_c.predict_proba(test_features[:,lr_s])[:,0]
probs_knn = knn_c.predict_proba(test_features[:,knn_s])[:,0]
all_probs = [probs_svm,probs_lr,probs_knn]

probs_features = all_probs[0].reshape(-1,1)
for i in range(1,len(all_probs)):
    probs_features = np.concatenate((probs_features,all_probs[i].reshape(-1,1)),axis=1)

In [17]:
joint_model = lr(penalty='l1').fit(probs_features[:30000,:], test_true_labels[:30000])
preds_lg = list(joint_model.predict(probs_features[30000:,:]))

acc = metrics.accuracy_score(list(map(int,test_true_labels[30000:])), list(map(int,preds_lg)))
f1 = metrics.f1_score(list(map(int,test_true_labels[30000:])), list(map(int,preds_lg)))
print('acc:',acc,'f1:',f1)

acc: 0.9712067292138467 f1: 0.9731976148888756


/home/jacob/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
# Joined forces
joined_DTree = DecisionTreeClassifier()

preds_test_svm = np.reshape(preds_svm,(len(preds_svm),1))
preds_test_lg = np.reshape(preds_lg,(len(preds_lg),1))
preds_test_knn = np.reshape(preds_knn,(len(preds_knn),1))
preds_test_dt = np.reshape(preds_dt,(len(preds_dt),1))
combined_preds = np.concatenate([preds_test_svm,preds_test_lg,preds_test_knn,preds_test_dt],axis=1)

joined_DTree.fit(combined_preds[0:50000,:], test_true_labels[0:50000])
preds_joined = joined_DTree.predict(combined_preds[50000:,:])

acc = metrics.accuracy_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
f1 = metrics.f1_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
print("acc:",acc,"f1:",f1)

acc: 0.970852428964253 f1: 0.9730234136409909


In [ ]:
# Which features are important?
# Histogram of the feature frequency for all selections that reached > 90% acc
frequency = [0]*total_num_features

num_good_preds = 0
min_acc = 0.93
for i,acc in enumerate(accs):
    if acc > min_acc:
        num_good_preds += 1
        for f in feature_selections[i]:
            frequency[f] += 1
#frequency = [freq/num_good_preds for freq in frequency]
print("")
print("number of classifiers: ",len(accs))
print("number of accs >",min_acc,": ",sum([1 for acc in accs if acc > min_acc]))
plt.figure()
plt.bar(x=range(len(frequency)),height=frequency)
plt.show()

In [116]:
# Adaboost DecisionTrees
from sklearn.ensemble import AdaBoostClassifier
selection =  [ 0,  1,  2,  4,  5,  6,  7, 10, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26]

ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4, min_samples_leaf = 1, min_samples_split = 2),
                         n_estimators=750,learning_rate=0.01)
ada.fit(train_features[:100000,selection],train_true_labels[:100000])
preds_ada = ada.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels)), list(map(int,preds_ada)))
f1 = metrics.f1_score(list(map(int,test_true_labels)), list(map(int,preds_ada)))
print("acc:",acc,"f1:",f1)
#0.9761527670935336

acc: 0.9740436709899852 f1: 0.9761527670935336


In [61]:
# ExtraTreesClassifier
#fiddle with n_estimators and min_samples_leaf

from sklearn.ensemble import ExtraTreesClassifier
# selection = [1, 2, 6, 13, 16, 20, 22, 23]
selection = [1, 2,  5,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

#add one to min_sample_leaf for full train_features (or change to 4)
extraTrees = ExtraTreesClassifier(n_estimators=750,max_depth=90,min_samples_split=10,min_samples_leaf=0.00001)

extraTrees.fit(competition_features[:-reserved,selection], all_true_labels[:-reserved])
preds_extra = list(extraTrees.predict(competition_features[-reserved:-l_competition,selection]))

acc = metrics.accuracy_score(list(map(int,all_true_labels[-reserved:-l_competition])), list(map(int,preds_extra)))
f1 = metrics.f1_score(list(map(int,all_true_labels[-reserved:-l_competition])), list(map(int,preds_extra)))
print('acc:',acc,'f1:',f1)

acc: 0.9645706780495813 f1: 0.9675674050918271


In [29]:
from sklearn.ensemble import RandomForestClassifier

#f1: 0.9803 750 estimators takes ~5 mins and ~10 mins with 1500 estimators
t0 = time.time()
selection = [2,  3,  5,  6,  7, 10, 11, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

randForest = RandomForestClassifier(n_estimators = 750, min_samples_split = 5, min_samples_leaf = 4, 
                                    max_depth = 60, n_jobs = -1)

randForest.fit(competition_features[:-reserved,selection], all_true_labels[:-label_reserved])
preds_randForest = randForest.predict(competition_features[-reserved:-l_competition,selection])
t1 = time.time()-t0
print(t1)

acc = metrics.accuracy_score(list(map(int,all_true_labels[-label_reserved:])), list(map(int,preds_randForest)))
f1 = metrics.f1_score(list(map(int,all_true_labels[-label_reserved:])), list(map(int,preds_randForest)))
print('acc:',acc,'f1:',f1)

41.762773513793945
acc: 0.9787 f1: 0.9803354362209571


In [218]:
selection = [2,  3,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
preds_randForest = randForest.predict(competition_features[-l_competition:,selection])

In [219]:
preds_rf = list(preds_randForest)

# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)
preds_rf = list(zip(range(len(competition_set)), preds_rf))

with open("rf_predictions1.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    for row in preds_rf:
        csv_out.writerow(row)
        
# with open('random_forest_model', 'wb') as file:
#         pickle.dump(randForest,file)

In [122]:
from xgboost.sklearn import XGBClassifier

selection = [2,22, 23, 25, 26, 7, 15, 21, 10, 5, 6,  16, 19, 27, 28, 29, 30, 31]
# selection = [i for i in range(16)]
# selection.extend([ 95, 102])

xgb = XGBClassifier(objective= 'binary:logistic', subsample = 0.8, colsample_bytree=0.8, learning_rate=0.01, 
                     max_depth=5, min_child_weight = 4, gamma=0, reg_lambda=2)

xgb.fit(train_features[:100000,selection],train_true_labels[:100000])
preds_xgb = xgb.predict(test_features[:,selection])
acc = metrics.accuracy_score(list(map(int,test_true_labels[:])), list(map(int,preds_xgb)))
f1 = metrics.f1_score(list(map(int,test_true_labels[:])), list(map(int,preds_xgb)))
print("acc:",acc,"f1:",f1)

acc: 0.9716795271712363 f1: 0.9739366926040642


In [25]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(objective= 'binary:logistic', subsample = 0.8, colsample_bytree=0.8, learning_rate=0.01, 
                     max_depth=5, min_child_weight = 4, gamma=0, reg_lambda=2)

In [12]:
selection = [0,2,7,8,9,13,16,19,20,21,22,23,24,25,26,27]
selection = [18,19,20,21]
xgb = XGBClassifier(objective= 'binary:logistic', subsample = 0.8, colsample_bytree=0.8, learning_rate=0.01, 
                     max_depth=5, min_child_weight = 4, gamma=0, reg_lambda=2)

xgb.fit(train_features[:25000,selection],train_true_labels[:25000])
preds_xgb = xgb.predict(train_features[25000:30000,selection])
acc = metrics.accuracy_score(list(map(int,train_true_labels[25000:30000])), list(map(int,preds_xgb)))
f1 = metrics.f1_score(list(map(int,train_true_labels[25000:30000])), list(map(int,preds_xgb)))
print("acc:",acc,"f1:",f1)

NameError: name 'XGBClassifier' is not defined

In [28]:
preds_xgb = list(preds_xgb)

# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)
preds_xgb = list(zip(range(len(competition_set)), preds_xgb))

with open("xgb_predictions.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    for row in preds_xgb:
        csv_out.writerow(row)

In [66]:
from sklearn.ensemble import VotingClassifier

#### NEEDS TO BE FIXED
preds_test_svm = np.reshape(preds_svm,(len(preds_svm),1))
preds_test_lg = np.reshape(preds_lg,(len(preds_lg),1))
preds_test_knn = np.reshape(preds_knn,(len(preds_knn),1))
preds_test_et = np.reshape(preds_extra,(len(preds_extra),1))
preds_test_xgb = np.reshape(preds_xgb,(len(preds_xgb),1))

combined_preds = np.concatenate([preds_test_svm,preds_test_lg,preds_test_knn,preds_test_et,preds_test_xgb],axis=1)

VotingClassifier.fit(combined_preds[0:50000,:], test_true_labels[0:50000])
preds_joined = VotingClassifier.predict(combined_preds[50000:,:])

acc = metrics.accuracy_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
f1 = metrics.f1_score(list(map(int,test_true_labels[50000:])), list(map(int,preds_joined)))
print("acc:",acc,"f1:",f1)

TypeError: fit() missing 1 required positional argument: 'y'

In [10]:
labels_int = []
for c in all_true_labels:
    labels_int.append(int(c))
labels_int = np.array(labels_int)

In [11]:
import lightgbm as light
import time

t0 = time.time()
selection = [2,  3,  5,  6,  7, 10, 11, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

#boosting_type='rf',subsample_freq=1, subsample = 0.3, colsample_bytree = 0.5
lgb = light.LGBMClassifier()

lgb.fit(competition_features[:-reserved,selection], labels_int[:-label_reserved])
preds_lgb = lgb.predict(competition_features[-reserved:-l_competition,selection])
t1 = time.time()-t0
print(t1)

acc = metrics.accuracy_score(list(map(int,labels_int[-label_reserved:])), list(map(int,preds_lgb)))
f1 = metrics.f1_score(list(map(int,labels_int[-label_reserved:])), list(map(int,preds_lgb)))
print('acc:',acc,'f1:',f1)

3.4225006103515625
acc: 0.9789666666666667 f1: 0.9805756503001385


In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

tuned_parameters = [{'num_leaves': [15,90],'min_child_samples': [50,3000],
                    'max_depth': [45,-1], 'learning_rate': [0.1,0.3], 'boosting_type': ['gbdt']}]

# [{'num_leaves': [15,30,60,90],'min_child_samples': [10,20,100,500,1500,3000],
#                     'max_depth': [3,5,15,50,100,-1]}]
selection = [2,  3,  5,  6,  7, 10, 11, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]

scores = ['f1_macro'] #'accuracy_score'
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(light.LGBMClassifier(), tuned_parameters, cv=2,
                       scoring='%s' % score, verbose=10, n_jobs=-1)
    clf.fit(competition_features[:-reserved,selection], labels_int[:-label_reserved])

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = labels_int[-label_reserved:], clf.predict(competition_features[-reserved:-l_competition,selection])
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for f1_macro

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  32 | elapsed:   20.7s remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   9 out of  32 | elapsed:   20.9s remaining:   53.3s
[Parallel(n_jobs=-1)]: Done  13 out of  32 | elapsed:   27.3s remaining:   39.9s
[Parallel(n_jobs=-1)]: Done  17 out of  32 | elapsed:   29.1s remaining:   25.7s
[Parallel(n_jobs=-1)]: Done  21 out of  32 | elapsed:   34.0s remaining:   17.8s
[Parallel(n_jobs=-1)]: Done  25 out of  32 | elapsed:   35.5s remaining:    9.9s
[Parallel(n_jobs=-1)]: Done  29 out of  32 | elapsed:   41.1s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:   48.1s finished


Best parameters set found on development set:

{'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 45, 'min_child_samples': 50, 'num_leaves': 90}

Grid scores on development set:

0.978 (+/-0.000) for {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 45, 'min_child_samples': 50, 'num_leaves': 15}
0.978 (+/-0.000) for {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 45, 'min_child_samples': 50, 'num_leaves': 90}
0.977 (+/-0.000) for {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 45, 'min_child_samples': 3000, 'num_leaves': 15}
0.978 (+/-0.000) for {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': 45, 'min_child_samples': 3000, 'num_leaves': 90}
0.978 (+/-0.000) for {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 50, 'num_leaves': 15}
0.978 (+/-0.000) for {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 50, 'num_leaves': 90}
0.977 (+/-0.000) for {'boosting_t

In [9]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import multiprocessing as mp
from sklearn.ensemble import RandomForestClassifier



randomForest = RandomForestClassifier(n_estimators = 500)
selection = [2,  3,  5,  6,  7, 10, 11, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]



min_samples_split = [2,5,10,20]
min_samples_leaf = [2,4,8,16]
max_depth = [30,60,120]


# Create the random grid
random_grid = {'min_samples_leaf': min_samples_leaf, 'min_samples_split': min_samples_split, 'max_depth': max_depth}

# Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator = randomForest, param_distributions = random_grid, n_iter = 25, cv = 2, verbose = 10, random_state=42, n_jobs = -1)

rf_random.fit(competition_features[:200000,selection], all_true_labels[:200000])

Fitting 2 folds for each of 25 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done   9 out of  50 | elapsed:  6.2min remaining: 28.2min
[Parallel(n_jobs=-1)]: Done  15 out of  50 | elapsed:  6.3min remaining: 14.7min
[Parallel(n_jobs=-1)]: Done  21 out of  50 | elapsed:  6.5min remaining:  8.9min
[Parallel(n_jobs=-1)]: Done  27 out of  50 | elapsed: 12.2min remaining: 10.4min
[Parallel(n_jobs=-1)]: Done  33 out of  50 | elapsed: 12.3min remaining:  6.3min
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed: 12.4min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed: 12.7min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 16.3min finished


KeyboardInterrupt: 

In [47]:
from scipy.stats import mode
top = np.argsort(rf_random.cv_results_['rank_test_score'])
# top[:5]
# xgb_random.best_index_
# mode([rf_random.cv_results_['params'][i]['bootstrap'] for i in top[:15]])
[ada_random.cv_results_['params'][i]['learning_rate'] for i in top[:10]]
#n_estimators: 750
#learning_rate: 0.01

#max_depth = 4
#min_samples_leaf = 1
#min_samples_split = 2
#min_weight_Fraction_leaf = 1e-05


AttributeError: 'RandomizedSearchCV' object has no attribute 'cv_results_'

In [111]:
import random

selection = [ 1,  7,  9, 10, 11, 15, 16, 19, 21, 22, 23]
selection.extend([6,25,26,27])

#37,38,40: 0.966
#42,43,50 .967
#53,54,55
#62,63,65
#67,68,66
#86,87,88
#106-108
def WRANDSEARCH(estimator, features, labels, baseline, num_features = 16, iterations = 50, 
                rate = 1.5, emp_avg = 0.968, emp_max = 0.975, emp_min = 0.96):
    
    cv0 = random.sample(range(len(features)),int(len(features)/2))
    cv1 = list(set(range(len(features)))-set(cv0))
    
    weights = np.ones(features.shape[1])
    tot_features = len(weights)
    
    #adjust weights so that by final iteration 50% of features selected are from high performing features and 50% are from the rest
    #as it stands this does not work properly for the reduction of probability
    cap = (num_features-len(baseline))/2
    evals_per_feature = iterations*(num_features-len(baseline))/tot_features
    alpha = rate*(cap-1)/evals_per_feature
    
    for i in range(iterations):
        if random.random()>0.5:
            selection = baseline
        else:
            removed = [random.choice(baseline), random.choice(baseline)]
            selection = [i for i in baseline if not i in removed]
            
        selection.extend(random.choices(range(27, features.shape[1]), weights = weights[27:], k = num_features-len(selection)))
        
        model = estimator.fit(features[cv0,:][:,selection],labels[cv0])
        preds = model.predict(features[cv1,:][:,selection])
        f1 = metrics.f1_score(list(map(int,labels[cv1])), list(map(int,preds)))
        model = estimator.fit(features[cv1,:][:,selection],labels[cv1])
        preds = model.predict(features[cv0,:][:,selection])
        f1 = (metrics.f1_score(list(map(int,labels[cv1])), list(map(int,preds))) + f1)/2
        
        if i==0:
            avg = 2*f1/3+emp_avg/3
            
        if f1-avg>0:
            for j in selection:
                weights[j] = weights[j]+alpha*(f1-avg)/(emp_max-avg)
        else:
            for j in selection:
                weights[j] = weights[j]*(1-0.35*(avg-f1)/(avg-emp_min))
            
        avg = (avg*3+f1)/4
        
        print(i, '/', iterations)
    
    return weights

In [112]:
from sklearn.linear_model import LogisticRegression as lr 

classifier = XGBClassifier(objective= 'binary:logistic', subsample = 0.8, colsample_bytree=0.8, learning_rate=0.01, 
                     max_depth=5, min_child_weight = 4, gamma=0, reg_lambda=2)

weights = WRANDSEARCH(classifier, train_sp[:60000], train_true_labels[:60000], [1,  2,  4,  6,  9, 11, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27], 
                      num_features = 21, iterations = 50)
weights

0 / 50
1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50


array([1.        , 1.89805278, 1.79589894, 1.        , 1.78291101,
       1.        , 1.87606643, 1.        , 1.        , 1.83137194,
       1.        , 1.90559243, 1.        , 1.        , 1.        ,
       1.        , 1.92520885, 1.93640245, 1.        , 1.        ,
       1.93564076, 1.93544767, 1.8534672 , 1.9270696 , 1.9209067 ,
       1.91657448, 1.82074365, 1.80740913, 1.        , 1.00834313,
       1.0032723 , 1.        , 1.00834313, 1.        , 1.00504387,
       1.        , 1.03667017, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.78414592, 1.        ,
       1.00625022, 1.        , 1.        , 1.        , 1.00049745,
       1.00040459, 1.        , 1.        , 1.        , 1.        ,
       1.00696388, 1.        , 1.9463327 , 1.        , 1.        ,
       1.        , 1.0042149 , 1.        , 1.        , 1.        ,
       1.00142482, 1.        , 1.00464836, 1.        , 1.     

In [132]:
np.where(weights>1.97)

(array([], dtype=int64),)

In [28]:
from sklearn.feature_selection import RFECV

estimator = RandomForestClassifier(n_estimators = 500, min_samples_split = 5, min_samples_leaf = 4, 
                                    max_depth = 60, n_jobs = -1)
selection = [2,  3,  5,  6,  7, 10, 11, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
# = ExtraTreesClassifier(n_estimators=500,max_depth=50,min_samples_split=10,min_samples_leaf=0.00001)
# = DecisionTreeClassifier()
# = lr(penalty='l1')
# = svm.LinearSVC(loss='hinge',max_iter=10000)

random.seed(0)

selector = RFECV(estimator, step=2, cv=2,min_features_to_select=15,verbose=10)
selector = selector.fit(competition_features[:150000,selection], all_true_labels[:150000])
selector.ranking_

Fitting estimator with 37 features.
Fitting estimator with 35 features.
Fitting estimator with 33 features.
Fitting estimator with 31 features.
Fitting estimator with 29 features.
Fitting estimator with 27 features.
Fitting estimator with 25 features.
Fitting estimator with 23 features.
Fitting estimator with 21 features.
Fitting estimator with 19 features.
Fitting estimator with 17 features.
Fitting estimator with 37 features.
Fitting estimator with 35 features.
Fitting estimator with 33 features.
Fitting estimator with 31 features.
Fitting estimator with 29 features.
Fitting estimator with 27 features.
Fitting estimator with 25 features.
Fitting estimator with 23 features.
Fitting estimator with 21 features.
Fitting estimator with 19 features.
Fitting estimator with 17 features.
Fitting estimator with 37 features.
Fitting estimator with 35 features.


array([1, 1, 1, 1, 1, 2, 3, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [29]:
np.where(selector.ranking_==1)

(array([ 0,  1,  2,  3,  4,  7,  8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]),)

In [14]:
import multi_func as mf

tocompute_features = {'succ_pred':False, 'Max_Sim':False, 'Citation_Check':False, 'node_degree':False, 'Reverse_Max_Sim':True}
p = mf.params(competition_graph, kdtree,l,node_dict,index_dict, to_do = tocompute_features, chunk_size = 1000)

In [15]:
import math
grouped_set = [all_set[2000*i:2000*(i+1)].tolist() for i in range(math.ceil(len(all_set)/2000))]

In [195]:
import multiprocessing as mp
import multi_func as mf
import time

pool = mp.Pool(24)

t0 = time.time() 
path_dict_list = pool.map(p.all_paths_noparams, grouped_set, chunksize = 10)

pool.close()

t1 = time.time()-t0
print(t1)

0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 5
0 / 

In [196]:
all_path_dict = dict()
for i in IDs:
    all_path_dict[i] = dict()
    
for d in path_dict_list:
    for e in d:
        for f in d[e]:
            all_path_dict[e][f] = d[e][f]

with open('all_path_dict', 'wb') as file:
        pickle.dump(all_path_dict,file)

In [ ]:
f = open('all_path_dict', 'rb')
all_path_dict = pickle.load(f)
f.close()

In [16]:
import multiprocessing as mp
import time
import multi_func as mf

pool = mp.Pool(24)

t0 = time.time() 
chunked_output = zip(*pool.map(p.by_chunk_noparams, grouped_set, chunksize = 10))

pool.close()

t1 = time.time()-t0
print(t1)
list_chunked_output = list(chunked_output)

# out1, out2, out3 = zip(*pool.map(calc_stuff, range(0, 10 * offset, offset)))

0 / 2
0 / 2
0 / 2
0 / 2
0 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
1 / 2
1 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
1 / 2
1 / 2
0 / 2
0 / 2
1 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
0 / 2
0 / 2
1 / 2
1 / 2
0 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
1 / 2
1 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
1 / 2
0 / 2
0 / 2
1 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
0 / 2
1 / 2
0 / 2
1 / 2
1 / 2
0 / 2
1 / 2
0 / 2
1 / 2
1 / 2
1 / 2
0 / 2
1 / 2
1 / 2
1 / 2
0 / 2
0 / 

In [20]:
features_to_create = ['Reverse_Max_Sim']
#(sp,ms,cc,nd)

insert_features_dict = dict()
for feat in features_to_create:
    insert_features_dict[feat] = []

for i in [4]:
    for feature_list in list_chunked_output[i]:
        insert_features_dict['Reverse_Max_Sim'].extend(feature_list)

In [21]:
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
    
for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./'+feat+'_allgraph')

Reverse_Max_Sim 648160


In [9]:
import time

features_to_create = ['Max_Sim']
insert_features_dict = dict()
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = training_set[:2200]
t0 = time.time()

sp,ms,cc,nd = pw.by_chunk(set_to_use,train_graph, kdtree,l,node_dict,index_dict, pairs_subset_edges=True, chunk_size=50,
                          to_do = {'succ_pred':False, 'Max_Sim':True, 'Citation_Check':False, 'node_degree':False}, k_cc=500, metric_ms='COS', n_ms=3)
insert_features_dict['Max_Sim'].extend(ms)
t1 = time.time()-t0

        
# for (name,value) in insert_features_dict.items():
#     print(name,len(value))
#     insert_features_dict[name] = to_feature_shape(value)

print(t1)
# for feat in features_to_create:
#     write_feature_to_disk(insert_features_dict[feat],'./features_train/'+feat+'_goldgraphfixed')

# features_to_create = ['node_degree']
# insert_features_dict = dict()
# for feat in features_to_create:
#     insert_features_dict[feat] = []
# set_to_use = testing_set

# sp,ms,cc,nd = pw.by_chunk(testing_set,train_graph, kdtree,l,node_dict,index_dict, pairs_subset_edges=False, chunk_size=1000,
#                           to_do = {'succ_pred':False, 'Max_Sim':False, 'Citation_Check':False, 'node_degree':True}, k_cc=500, metric_ms='COS', n_ms=3)
# insert_features_dict['node_degree'].extend(nd)
        
# for (name,value) in insert_features_dict.items():
#     print(name,len(value))
#     insert_features_dict[name] = to_feature_shape(value)
    
# for feat in features_to_create:
#     write_feature_to_disk(insert_features_dict[feat],'./features_test/'+feat+'_goldgraphfixed')


0 / 44
1 / 44
2 / 44
3 / 44
4 / 44
5 / 44
6 / 44
7 / 44
8 / 44
9 / 44
10 / 44
11 / 44
12 / 44
13 / 44
14 / 44
15 / 44
16 / 44
17 / 44
18 / 44
19 / 44
20 / 44
21 / 44
22 / 44
23 / 44
24 / 44
25 / 44
26 / 44
27 / 44
28 / 44
29 / 44
30 / 44
31 / 44
32 / 44
33 / 44
34 / 44
35 / 44
36 / 44
37 / 44
38 / 44
39 / 44
40 / 44
41 / 44
42 / 44
43 / 44
68.4776840209961


In [35]:
############################################
###  Construct features on TRAINING_SET  ###
############################################


train_true_labels = []
features_to_create = [#'peer_popularity', 'max_sim', 'edge_check', 'title_sim', 'citation_check', 'node_degree','succ_pred', 'N_LSA_distance']
#'peer_popularity', 'max_sim', 'edge_check', 'title_sim', 'citation_check', 'node_degree','succ_pred', 'N_LSA_distance'
    
# Where to insert created features
insert_features_dict = train_features_dict
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = training_set
title_sim_ones = []
title_sim_zeros = []

for i,triple in enumerate(set_to_use):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    
    source_info = node_info[index_source]
    target_info = node_info[index_target]

    # convert to lowercase and tokenize
#     source_title = source_info[2].lower().split(" ")
    # remove stopwords
#     source_title = [token for token in source_title if token not in stpwds]
#     source_title = [stemmer.stem(token) for token in source_title]

#     target_title = target_info[2].lower().split(" ")
#     target_title = [token for token in target_title if token not in stpwds]
#     target_title = [stemmer.stem(token) for token in target_title]

#     source_auth = source_info[3].split(",")
#     target_auth = target_info[3].split(",") 
    
    # Creating features
    # Baseline #
    #overlap_title = len(set(source_title).intersection(set(target_title)))
    #insert_features_dict["overlap_title"].append(overlap_title)
    #temp_diff = int(source_info[1]) - int(target_info[1])
    #insert_features_dict["temp_diff"].append(temp_diff)
    #comm_auth = len(set(source_auth).intersection(set(target_auth)))
    #insert_features_dict["comm_auth"].append(comm_auth)

#     peer_pop = pw.peer_popularity(train_graph,source,target)
#     insert_features_dict["peer_popularity"].append(peer_pop)

#     max_sim = pw.Max_Sim(source,target,l,train_graph,node_dict)
#     insert_features_dict["max_sim"].append(max_sim)

#     edge_check = pw.edge_check(source,target,train_graph)
#     insert_features_dict["edge_check"].append(edge_check)

    #LSA_dist = pw.LSA_distance(source,target,node_dict,l)
    #insert_features_dict["LSA_distance"].append(LSA_dist)
    
#     title_weighted = title_sim[index_source,index_target]
#     insert_features_dict["title_sim"].append(title_weighted)
#     if(int(triple[2]) == 0):
#         title_sim_zeros.append(title_weighted)
#     else:
#         title_sim_ones.append(title_weighted)
#     citation_check = pw.Citation_Check(source,target,kdtree,l,train_graph,node_dict,index_dict,k=500)
#     insert_features_dict["citation_check"].append(citation_check)

#     degree = pw.node_degree(source,target,train_graph)
#     insert_features_dict["node_degree"].append(degree)

    succ_pred = pw.succ_pred(source,target,train_graph)
    insert_features_dict["succ_pred"].append(succ_pred)
    
#     N_LSA_dist = pw.LSA_distance(source,target,node_dict,l_ngrams)
#     insert_features_dict["N_LSA_distance"].append(N_LSA_dist)

#     temporal_fit = pw.temp_fit(source,target,train_graph,node_dict,publication_years)
#     insert_features_dict["temporal_fit"].append(temporal_fit)
    
#     path_length = pw.path_length(source, target, training_paths_dict)
#     insert_features_dict["path_length"].append(path_length)
    
#     train_true_labels.append(triple[2])

    if i%1000==0:
        print(i,"/",len(set_to_use))

# Reshape features into np column arrays, one row per node pair
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
    
for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./features_train/'+feat+'_goldgraphfixed')
        
# Concatenate all features
# feats_train = np.concatenate([feat for feat in insert_features_dict.values()])
# train_true_labels = np.array(train_true_labels)

0 / 554602
100 / 554602
200 / 554602
300 / 554602
400 / 554602


KeyboardInterrupt: 

In [144]:
file_path = './features_train/'+'max_sim'+'_reducedgraph'
try:
    this_feat = read_feature(file_path)
except:
    print(1)

In [107]:
testing_paths_dict = prep.all_paths(testing_set,train_graph,pairs_subset_edges=False)

0 / 175
20 / 175
40 / 175
60 / 175
80 / 175
100 / 175
120 / 175
140 / 175
160 / 175


In [7]:
###########################################
###  Construct features on TESTING_SET  ###
###########################################

features_to_create = ['succ_pred']
#'peer_popularity', 'max_sim', 'edge_check', 'title_sim', 'citation_check','node_degree','succ_pred','N_LSA_distance'

# Where to insert created features
insert_features_dict = test_features_dict
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = testing_set

test_true_labels = []
for i,triple in enumerate(set_to_use):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    
    source_info = node_info[index_source]
    target_info = node_info[index_target]

    # convert to lowercase and tokenize
#     source_title = source_info[2].lower().split(" ")
    # remove stopwords
#     source_title = [token for token in source_title if token not in stpwds]
#     source_title = [stemmer.stem(token) for token in source_title]

#     target_title = target_info[2].lower().split(" ")
#     target_title = [token for token in target_title if token not in stpwds]
#     target_title = [stemmer.stem(token) for token in target_title]

#     source_auth = source_info[3].split(",")
#     target_auth = target_info[3].split(",") 
    
    
    # Creating features
    # Baseline #
    #overlap_title = len(set(source_title).intersection(set(target_title)))
    #insert_features_dict["overlap_title"].append(overlap_title)
    #temp_diff = int(source_info[1]) - int(target_info[1])
    #insert_features_dict["temp_diff"].append(temp_diff)
    #comm_auth = len(set(source_auth).intersection(set(target_auth)))
    #insert_features_dict["comm_auth"].append(comm_auth)

#     peer_pop = pw.peer_popularity(train_graph,source,target)
#     insert_features_dict["peer_popularity"].append(peer_pop)

#     max_sim = pw.Max_Sim(source,target,l,train_graph,node_dict)
#     insert_features_dict["max_sim"].append(max_sim)

#     edge_check = pw.edge_check(source,target,train_graph)
#     insert_features_dict["edge_check"].append(edge_check)

    #LSA_dist = pw.LSA_distance(source,target,node_dict,l)
    #insert_features_dict["LSA_distance"].append(LSA_dist)
    
#     title_weighted = title_sim[index_source,index_target]
#     insert_features_dict["title_sim"].append(title_weighted)
    
#     citation_check = pw.Citation_Check(source,target,kdtree,l,train_graph,node_dict,index_dict,k=500)
#     insert_features_dict["citation_check"].append(citation_check)
    
    succ_pred = pw.succ_pred(source,target,train_graph)
    insert_features_dict["succ_pred"].append(succ_pred)
    
#     degree = pw.node_degree(source,target,train_graph)
#     insert_features_dict["node_degree"].append(degree)
    
#     N_LSA_dist = pw.LSA_distance(source,target,node_dict,l_ngrams)
#     insert_features_dict["N_LSA_distance"].append(N_LSA_dist)

#     temporal_fit = pw.temp_fit(source,target,train_graph,node_dict,publication_years)
#     insert_features_dict["temporal_fit"].append(temporal_fit)
    
#     path_length = pw.path_length(source, target, testing_paths_dict)
#     insert_features_dict["path_length"].append(path_length)
    
#     test_true_labels.append(triple[2])

    if i%1000==0:
        print(i,"/",len(set_to_use))

# Reshape features into np column arrays, one row per node pair
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
        
# Concatenate all features
# feats_test = np.concatenate([feat for feat in insert_features_dict.values()])
# test_true_labels = np.array(test_true_labels)

for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./features_test/'+feat+'_goldgraphfixed')

0 / 60910
1000 / 60910
2000 / 60910
3000 / 60910
4000 / 60910
5000 / 60910
6000 / 60910
7000 / 60910
8000 / 60910
9000 / 60910
10000 / 60910
11000 / 60910
12000 / 60910
13000 / 60910
14000 / 60910
15000 / 60910
16000 / 60910
17000 / 60910
18000 / 60910
19000 / 60910
20000 / 60910
21000 / 60910
22000 / 60910
23000 / 60910
24000 / 60910
25000 / 60910
26000 / 60910
27000 / 60910
28000 / 60910
29000 / 60910
30000 / 60910
31000 / 60910
32000 / 60910
33000 / 60910
34000 / 60910
35000 / 60910
36000 / 60910
37000 / 60910
38000 / 60910
39000 / 60910
40000 / 60910
41000 / 60910
42000 / 60910
43000 / 60910
44000 / 60910
45000 / 60910
46000 / 60910
47000 / 60910
48000 / 60910
49000 / 60910
50000 / 60910
51000 / 60910
52000 / 60910
53000 / 60910
54000 / 60910
55000 / 60910
56000 / 60910
57000 / 60910
58000 / 60910
59000 / 60910
60000 / 60910
succ_pred 60910


In [6]:
node_info_array = np.array(node_info)
publication_years = node_info_array[:,1]

In [197]:
###########################################
###  Construct features on COMPETITION_SET  ###
###########################################

features_to_create = ['path_length']
# "overlap_title",
#                  "comm_auth",
#                  "temp_diff",
#                 "citation_check",
#                 "max_sim",
#                 "peer_popularity",
#                 "edge_check",
#                 "LSA_distance",
#                 "node_degree",
#                 "succ_pred",
#                 "title_sim",
#                 "temporal_fit",
#                 "N_LSA_distance",                 

# Where to insert created features
insert_features_dict = dict()
for feat in features_to_create:
    insert_features_dict[feat] = []
set_to_use = all_set

for i,triple in enumerate(set_to_use):
    source = triple[0]
    target = triple[1]
    index_source = node_dict[source]
    index_target = node_dict[target]
    
    source_info = node_info[index_source]
    target_info = node_info[index_target]

#     #convert to lowercase and tokenize
#     source_title = source_info[2].lower().split(" ")
#     #remove stopwords
#     source_title = [token for token in source_title if token not in stpwds]
#     source_title = [stemmer.stem(token) for token in source_title]

#     target_title = target_info[2].lower().split(" ")
#     target_title = [token for token in target_title if token not in stpwds]
#     target_title = [stemmer.stem(token) for token in target_title]

#     source_auth = source_info[3].split(",")
#     target_auth = target_info[3].split(",") 

# sp, ms, cc = pw.by_chunk(competition_set,train_graph, kdtree,l,node_dict,index_dict, pairs_subset_edges=False, chunk_size=1000, k_cc=500, metric_ms='COS', n_ms=3)
# insert_features_dict['citation_check'].extend(cc)
# insert_features_dict['max_sim'].extend(ms)
# insert_features_dict['succ_pred'].extend(sp)
    
    
    # Creating features
    # Baseline #
#     overlap_title = len(set(source_title).intersection(set(target_title)))
#     insert_features_dict["overlap_title"].append(overlap_title)
#     temp_diff = int(source_info[1]) - int(target_info[1])
#     insert_features_dict["temp_diff"].append(temp_diff)
#     comm_auth = len(set(source_auth).intersection(set(target_auth)))
#     insert_features_dict["comm_auth"].append(comm_auth)

#     peer_pop = pw.peer_popularity(competition_graph,source,target)
#     insert_features_dict["peer_popularity"].append(peer_pop)

#     edge_check = pw.edge_check(source,target,competition_graph)
#     insert_features_dict['edge_check'].append(0)

#     LSA_dist = pw.LSA_distance(source,target,node_dict,l)
#     insert_features_dict["LSA_distance"].append(LSA_dist)
    
#     ts = title_sim[index_source,index_target]
#     insert_features_dict["title_sim"].append(ts)
    
#     temporal_fit = pw.temp_fit(source,target,competition_graph,node_dict,publication_years)
#     insert_features_dict["temporal_fit"].append(temporal_fit)
    
    
    path_length = pw.path_length(source, target, all_path_dict)
    insert_features_dict["path_length"].append(path_length)
    
#     N_LSA_dist = pw.LSA_distance(source,target,node_dict,l_ngrams)
#     insert_features_dict["N_LSA_distance"].append(N_LSA_dist)

    if i%1000==0:
        print(i,"/",len(set_to_use))

# Reshape features into np column arrays, one row per node pair
for (name,value) in insert_features_dict.items():
    print(name,len(value))
    insert_features_dict[name] = to_feature_shape(value)
        
# Concatenate all features
# feats_test = np.concatenate([feat for feat in insert_features_dict.values()])
# test_true_labels = np.array(test_true_labels)

for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./'+feat+'_allgraph')

0 / 648160
1000 / 648160
2000 / 648160
3000 / 648160
4000 / 648160
5000 / 648160
6000 / 648160
7000 / 648160
8000 / 648160
9000 / 648160
10000 / 648160
11000 / 648160
12000 / 648160
13000 / 648160
14000 / 648160
15000 / 648160
16000 / 648160
17000 / 648160
18000 / 648160
19000 / 648160
20000 / 648160
21000 / 648160
22000 / 648160
23000 / 648160
24000 / 648160
25000 / 648160
26000 / 648160
27000 / 648160
28000 / 648160
29000 / 648160
30000 / 648160
31000 / 648160
32000 / 648160
33000 / 648160
34000 / 648160
35000 / 648160
36000 / 648160
37000 / 648160
38000 / 648160
39000 / 648160
40000 / 648160
41000 / 648160
42000 / 648160
43000 / 648160
44000 / 648160
45000 / 648160
46000 / 648160
47000 / 648160
48000 / 648160
49000 / 648160
50000 / 648160
51000 / 648160
52000 / 648160
53000 / 648160
54000 / 648160
55000 / 648160
56000 / 648160
57000 / 648160
58000 / 648160
59000 / 648160
60000 / 648160
61000 / 648160
62000 / 648160
63000 / 648160
64000 / 648160
65000 / 648160
66000 / 648160
67000 / 

551000 / 648160
552000 / 648160
553000 / 648160
554000 / 648160
555000 / 648160
556000 / 648160
557000 / 648160
558000 / 648160
559000 / 648160
560000 / 648160
561000 / 648160
562000 / 648160
563000 / 648160
564000 / 648160
565000 / 648160
566000 / 648160
567000 / 648160
568000 / 648160
569000 / 648160
570000 / 648160
571000 / 648160
572000 / 648160
573000 / 648160
574000 / 648160
575000 / 648160
576000 / 648160
577000 / 648160
578000 / 648160
579000 / 648160
580000 / 648160
581000 / 648160
582000 / 648160
583000 / 648160
584000 / 648160
585000 / 648160
586000 / 648160
587000 / 648160
588000 / 648160
589000 / 648160
590000 / 648160
591000 / 648160
592000 / 648160
593000 / 648160
594000 / 648160
595000 / 648160
596000 / 648160
597000 / 648160
598000 / 648160
599000 / 648160
600000 / 648160
601000 / 648160
602000 / 648160
603000 / 648160
604000 / 648160
605000 / 648160
606000 / 648160
607000 / 648160
608000 / 648160
609000 / 648160
610000 / 648160
611000 / 648160
612000 / 648160
613000 /

In [18]:
for feat in features_to_create:
    write_feature_to_disk(insert_features_dict[feat],'./features_test/'+feat+'_goldgraphfixed')